In [284]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stop_words_set = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

In [ ]:
repo = "pytorch"
df = pd.read_pickle(f"sampled_bugs/{repo}.pkl")

,number,title,url,bodyText,body,comments,createdAt,closedAt,assignees,text
1,145967,lintrunner fakily times out while running dmypy,https://github.com/pytorch/pytorch/issues/145967,🐛 Describe the bug\nError normally looks like:...,### 🐛 Describe the bug\n\nError normally looks...,_user_comment_: Trying to find first failure:\...,2025-01-29T20:50:39Z,2025-01-30T17:29:17Z,{'nodes': [{'login': 'ZainRizvi'}]},lintrunner fakily times out while running dmyp...
2,145825,[Inductor-CPU] Codegened flex attention kernel...,https://github.com/pytorch/pytorch/issues/145825,🐛 Describe the bug\nProblem statement\nCodegen...,### 🐛 Describe the bug\n\n### Problem statemen...,_user_comment_: If the environment variable `T...,2025-01-28T06:28:41Z,2025-01-29T20:54:49Z,{'nodes': [{'login': 'sanchitintel'}]},[Inductor-CPU] Codegened flex attention kernel...
3,145735,[MPSInductor] Compiled lgamma produces differe...,https://github.com/pytorch/pytorch/issues/145735,🐛 Describe the bug\nRunning this on my M4\n% p...,### 🐛 Describe the bug\n\nRunning this on my M...,_user_comment_: Glitch happens on that line\nh...,2025-01-27T15:58:09Z,2025-01-28T04:38:38Z,{'nodes': [{'login': 'malfet'}]},[MPSInductor] Compiled lgamma produces differe...
4,145734,Workaround for unsupported return of multiple ...,https://github.com/pytorch/pytorch/issues/145734,🐛 Describe the bug\nTrying to onnx.export a nn...,### 🐛 Describe the bug\n\nTrying to `onnx.expo...,_user_comment_: The warning in the documentati...,2025-01-27T15:42:57Z,2025-01-29T04:13:26Z,{'nodes': [{'login': 'justinchuby'}]},Workaround for unsupported return of multiple ...
5,145704,Bot spamming on PR trying to deploy to upload-...,https://github.com/pytorch/pytorch/issues/145704,"🐛 Describe the bug\nReported by @ezyang, for e...","### 🐛 Describe the bug\n\nReported by @ezyang,...",_user_comment_: +1 _user_comment_: I see those...,2025-01-26T16:25:43Z,2025-01-28T17:46:34Z,{'nodes': [{'login': 'huydhn'}]},Bot spamming on PR trying to deploy to upload-...
...,...,...,...,...,...,...,...,...,...,...
6678,30845,Many functions fail gradcheck on ROCm,https://github.com/pytorch/pytorch/issues/30845,In #30821 I started running a bunch of gradche...,In #30821 I started running a bunch of gradche...,No comments for this issue.,2019-12-05T21:56:55Z,2020-12-09T08:46:17Z,{'nodes': []},Many functions fail gradcheck on ROCm\nIn #308...
6679,29055,TorchScript does not preserve string behavior ...,https://github.com/pytorch/pytorch/issues/29055,"🐛 Bug\nIf a function calls len(s), it works di...","## 🐛 Bug\r\n\r\nIf a function calls `len(s)`, ...",_user_comment_: Closing until we have a plan f...,2019-11-01T19:16:49Z,2020-12-18T19:15:34Z,{'nodes': [{'login': 'ansley'}]},TorchScript does not preserve string behavior ...
6680,27356,ROCm tests are run on the default stream,https://github.com/pytorch/pytorch/issues/27356,🐛 Bug\nROCm tests are run exclusively on the d...,## 🐛 Bug\r\n\r\nROCm tests are run exclusively...,No comments for this issue.,2019-10-04T02:04:42Z,2020-12-10T18:00:15Z,{'nodes': []},ROCm tests are run on the default stream\n🐛 Bu...
6685,24504,Circular padding doesn't work when the 'right'...,https://github.com/pytorch/pytorch/issues/24504,Reproduction:\nimport torch\nimport torch.nn.f...,Reproduction:\r\n```\r\nimport torch\r\nimport...,_user_comment_: I just hit the same bug. _user...,2019-08-16T18:30:21Z,2020-12-06T03:59:23Z,{'nodes': []},Circular padding doesn't work when the 'right'...


In [294]:
def mask_code_snippets(text):
    block_regex = r'```.*?```'
    inline_regex = r'`.*?`'

    all_code_snippets = re.findall(block_regex, text, flags=re.DOTALL) + re.findall(inline_regex, text, flags=re.DOTALL)
    snippets_map = {}
    masked_text = text

    for i, code in enumerate(all_code_snippets):
        placeholder = f"_code_snippet_{i}_"
        cleaned_code = code.strip('`').strip()
        cleaned_code = re.sub(r'[\r\n]+', ' ', cleaned_code).strip()
        snippets_map[placeholder] = cleaned_code
        masked_text = masked_text.replace(code, placeholder)
    masked_text = masked_text.lower()
    return masked_text, snippets_map

def mask_urls(text):
    url_regex = r'(http[s]?://\S+|www\.\S+)'

    urls = re.findall(url_regex, text)
    url_map = {}
    masked_text = text

    for i, url in enumerate(urls):
        placeholder = f"_url_{i}_"
        url_map[placeholder] = url
        masked_text = masked_text.replace(url, placeholder)
    return masked_text, url_map
        

def preprocess_text(text):
    masked_text, snippets_map = mask_code_snippets(text)
    masked_text, urls_map = mask_urls(masked_text)
    tokens = re.findall(r'[a-z]+|\d+|_code_snippet_\d+_|_url_\d+_', masked_text)
    return ' '.join([
        snippets_map[token] if token.startswith("_code_snippet_") else
        stemmer.stem(token)
        for token in tokens
        if token.startswith("_code_snippet_") or token.startswith("_url_") or token not in stop_words_set
    ])


In [ ]:
df['description_tokens'] = df['text'].apply(preprocess_text)
df['comments_tokens'] = df['comments'].apply(preprocess_text)
df['complete_text'] = (df['description_tokens'] + ' ' + df['comments_tokens'])

,index,number,title,url,bodyText,body,comments,createdAt,closedAt,assignees,text,description_tokens,comments_tokens,complete_text
0,1,145967,lintrunner fakily times out while running dmypy,https://github.com/pytorch/pytorch/issues/145967,🐛 Describe the bug\nError normally looks like:...,### 🐛 Describe the bug\n\nError normally looks...,_user_comment_: Trying to find first failure:\...,2025-01-29T20:50:39Z,2025-01-30T17:29:17Z,{'nodes': [{'login': 'ZainRizvi'}]},lintrunner fakily times out while running dmyp...,lintrunn fakili time run dmypi describ bug err...,user comment tri find first failur _url_0_ hap...,lintrunn fakili time run dmypi describ bug err...
1,2,145825,[Inductor-CPU] Codegened flex attention kernel...,https://github.com/pytorch/pytorch/issues/145825,🐛 Describe the bug\nProblem statement\nCodegen...,### 🐛 Describe the bug\n\n### Problem statemen...,_user_comment_: If the environment variable `T...,2025-01-28T06:28:41Z,2025-01-29T20:54:49Z,{'nodes': [{'login': 'sanchitintel'}]},[Inductor-CPU] Codegened flex attention kernel...,inductor cpu codegen flex attent kernel appear...,user comment environ variabl TORCHINDUCTOR_CPP...,inductor cpu codegen flex attent kernel appear...
2,3,145735,[MPSInductor] Compiled lgamma produces differe...,https://github.com/pytorch/pytorch/issues/145735,🐛 Describe the bug\nRunning this on my M4\n% p...,### 🐛 Describe the bug\n\nRunning this on my M...,_user_comment_: Glitch happens on that line\nh...,2025-01-27T15:58:09Z,2025-01-28T04:38:38Z,{'nodes': [{'login': 'malfet'}]},[MPSInductor] Compiled lgamma produces differe...,mpsinductor compil lgamma produc differ result...,user comment glitch happen line _url_0_ look l...,mpsinductor compil lgamma produc differ result...
3,4,145734,Workaround for unsupported return of multiple ...,https://github.com/pytorch/pytorch/issues/145734,🐛 Describe the bug\nTrying to onnx.export a nn...,### 🐛 Describe the bug\n\nTrying to `onnx.expo...,_user_comment_: The warning in the documentati...,2025-01-27T15:42:57Z,2025-01-29T04:13:26Z,{'nodes': [{'login': 'justinchuby'}]},Workaround for unsupported return of multiple ...,workaround unsupport return multipl tensor tor...,user comment warn document seem outdat updat s...,workaround unsupport return multipl tensor tor...
4,5,145704,Bot spamming on PR trying to deploy to upload-...,https://github.com/pytorch/pytorch/issues/145704,"🐛 Describe the bug\nReported by @ezyang, for e...","### 🐛 Describe the bug\n\nReported by @ezyang,...",_user_comment_: +1 _user_comment_: I see those...,2025-01-26T16:25:43Z,2025-01-28T17:46:34Z,{'nodes': [{'login': 'huydhn'}]},Bot spamming on PR trying to deploy to upload-...,bot spam pr tri deploy upload benchmark result...,user comment 1 user comment see lot though nec...,bot spam pr tri deploy upload benchmark result...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3612,6678,30845,Many functions fail gradcheck on ROCm,https://github.com/pytorch/pytorch/issues/30845,In #30821 I started running a bunch of gradche...,In #30821 I started running a bunch of gradche...,No comments for this issue.,2019-12-05T21:56:55Z,2020-12-09T08:46:17Z,{'nodes': []},Many functions fail gradcheck on ROCm\nIn #308...,mani function fail gradcheck rocm 30821 start ...,comment issu,mani function fail gradcheck rocm 30821 start ...
3613,6679,29055,TorchScript does not preserve string behavior ...,https://github.com/pytorch/pytorch/issues/29055,"🐛 Bug\nIf a function calls len(s), it works di...","## 🐛 Bug\r\n\r\nIf a function calls `len(s)`, ...",_user_comment_: Closing until we have a plan f...,2019-11-01T19:16:49Z,2020-12-18T19:15:34Z,{'nodes': [{'login': 'ansley'}]},TorchScript does not preserve string behavior ...,torchscript preserv string behavior string uni...,user comment close plan full unicod support,torchscript preserv string behavior string uni...
3614,6680,27356,ROCm tests are run on the default stream,https://github.com/pytorch/pytorch/issues/27356,🐛 Bug\nROCm tests are run ex

In [302]:
df = df.reset_index() 
df.to_pickle(f"processed_issues/{repo}_processed.pkl")